In [1]:
using Laplacians

objc[42976]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[42976]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[42976]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[42976]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [26]:
include("treeSolver.jl")
include("IO.jl")
include("generators.jl")
include("toHighPrecision.jl")
include("verbosePCG.jl")

pcgVBLASMatNorm (generic function with 1 method)

In [27]:
pwd()

"/Users/serbanstan/git/TreePCG/julia"

In [28]:
a = readFromFile("../graphs/se_1/graph.mtx");

In [29]:
tree = readFromFile("../graphs/se_1/tree.mtx");

In [30]:
x = readFromFile("../graphs/se_1/x.vec");

In [31]:
# b = readFromFile("../graphs/se_1/b.vec");
b = lap(a) * x;

In [32]:
norm(lap(a) * x - b) / norm(b)

0.0

In [33]:
la = lap(a);

In [34]:
ts = treeSolver(tree)

F = function(b)
    return ts(b - mean(b))
end

(anonymous function)

In [35]:
@time myx64 = pcgVMatNorm(la, b, F, x, tol=1e-3, maxits=100000, verbose=true);
norm(lap(a) * myx64 - b) / norm(b)

Working on iteration 100 with error 269.7263227231269
Working on iteration 200 with error 99.34983753313102
Working on iteration 300 with error 60.182238757001244
Working on iteration 400 with error 47.507408397298704
Working on iteration 500 with error 38.745691056433344
Working on iteration 600 with error 32.132773996369586
Working on iteration 700 with error 27.880454612970425
Working on iteration 800 with error 24.698907411339956
Working on iteration 900 with error 22.62545988819161
Working on iteration 1000 with error 21.000213402591864
Working on iteration 1100 with error 19.15289308136822
Working on iteration 1200 with error 17.40701054537061
Working on iteration 1300 with error 16.603202325352896
Working on iteration 1400 with error 15.92599905969587
Working on iteration 1500 with error 15.3180699076338
Working on iteration 1600 with error 14.983843490126205
Working on iteration 1700 with error 14.620776794488817
Working on iteration 1800 with error 14.298098320645243
Working o

5.341284933996111e-10

In [37]:
dbg = []

for precision in [64,128,1024]
    hp = precision
    set_bigfloat_precision(hp)
    laHP = toHighPrecision(la; precision=hp)
    treeHP = toHighPrecision(tree; precision=hp)
    xHP = toHighPrecision(x; precision=hp);
    bHP = toHighPrecision(b; precision=hp);
    tsHP = treeSolver(treeHP)

    FHP = function(b)
        return tsHP(b - mean(b))
    end
    
    push!(dbg, "======= $(hp) bit precision")
    @time myxHP = pcgVMatNorm(laHP, bHP, FHP, xHP, tol=1e-3, maxits=100000, verbose=true);
    norm(laHP * myxHP - bHP) / norm(bHP)
    push!(dbg, "")
    push!(dbg, "")
    
end

Working on iteration 100 with error 2.57354271870849476922e+02
Working on iteration 200 with error 9.63160023344863077677e+01
Working on iteration 300 with error 5.71301616074166983827e+01
Working on iteration 400 with error 4.43248785079218231529e+01
Working on iteration 500 with error 3.56007947681027436954e+01
Working on iteration 600 with error 2.95267170897357084224e+01
Working on iteration 700 with error 2.59693377951084328186e+01
Working on iteration 800 with error 2.29197458054033311858e+01
Working on iteration 900 with error 2.09477608931756584098e+01
Working on iteration 1000 with error 1.90519844367259991186e+01
Working on iteration 1100 with error 1.70891487427500564887e+01
Working on iteration 1200 with error 1.62740423959879157451e+01
Working on iteration 1300 with error 1.54342291601545240761e+01
Working on iteration 1400 with error 1.50195097392184077631e+01
Working on iteration 1500 with error 1.46525738141534738954e+01
Working on iteration 1600 with error 1.4265958235

LoadError: LoadError: InterruptException:
while loading In[37], in expression starting on line 3

In [38]:
writeToFile("../graphs/se_1/_log_julia.txt", dbg)